## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [17]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("./WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Guerrero Negro,MX,27.98,-114.06,65.62,74,20,12.26,few clouds
1,1,Cherskiy,RU,68.75,161.30,-2.52,99,55,3.74,broken clouds
2,2,Jamestown,US,42.10,-79.24,57.02,78,75,6.91,broken clouds
3,3,Moth,IN,25.72,78.95,70.83,53,71,4.94,broken clouds
4,4,Porgera,PG,-5.47,143.20,56.35,69,75,2.55,broken clouds


In [20]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Vacation Criteria
# Input statements
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Create df that contains all cities that meet temperature criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Luanda,AO,-8.84,13.23,77.00,88,75,5.75,broken clouds
12,12,Mancio Lima,BR,-7.61,-72.90,76.82,51,45,2.84,scattered clouds
15,15,Poum,NC,-20.23,164.02,79.52,75,3,5.28,clear sky
16,16,Bengkulu,ID,-3.80,102.27,78.03,80,100,3.09,overcast clouds
21,21,Hithadhoo,MV,-0.60,73.08,81.37,76,34,12.66,scattered clouds
22,22,Flinders,AU,-34.58,150.86,76.17,68,9,7.00,clear sky
31,31,Meulaboh,ID,4.14,96.13,78.33,82,98,4.41,overcast clouds
35,35,Vanimo,PG,-2.67,141.30,87.01,63,99,2.51,overcast clouds
37,37,Bonoua,CI,5.27,-3.60,78.42,91,20,2.86,few clouds
44,44,Piacabucu,BR,-10.41,-36.43,75.78,86,38,13.24,scattered clouds


In [25]:
len(preferred_cities_df)

172

In [22]:
# 4a. Determine if there are any empty rows.
# Look for null values.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [27]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities_df=preferred_cities_df.dropna()

In [28]:
len(clean_travel_cities_df)

172

In [30]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = " "
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Luanda,AO,77.00,broken clouds,-8.84,13.23,
12,Mancio Lima,BR,76.82,scattered clouds,-7.61,-72.90,
15,Poum,NC,79.52,clear sky,-20.23,164.02,
16,Bengkulu,ID,78.03,overcast clouds,-3.80,102.27,
21,Hithadhoo,MV,81.37,scattered clouds,-0.60,73.08,
22,Flinders,AU,76.17,clear sky,-34.58,150.86,
31,Meulaboh,ID,78.33,overcast clouds,4.14,96.13,
35,Vanimo,PG,87.01,overcast clouds,-2.67,141.30,
37,Bonoua,CI,78.42,few clouds,5.27,-3.60,
44,Piacabucu,BR,75.78,scattered clouds,-10.41,-36.43,


In [31]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [33]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!=" "]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Luanda,AO,77.00,broken clouds,-8.84,13.23,EPIC SANA Luanda
12,Mancio Lima,BR,76.82,scattered clouds,-7.61,-72.90,Vila Pousada
16,Bengkulu,ID,78.03,overcast clouds,-3.80,102.27,Grage Hotel Bengkulu
21,Hithadhoo,MV,81.37,scattered clouds,-0.60,73.08,Scoop Guest House
22,Flinders,AU,76.17,clear sky,-34.58,150.86,Shellharbour Resort & Conference Centre


In [39]:
# 8a. Create the output File (CSV)
output_data_file="./clean_hotel_df.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [40]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [65]:
# 11a. Add a marker layer for each city to the map. 
# Assign variable for pop-up marker to be placed on each city.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Maximum Temperature Heatmap from a hotel_df with pop-up markers.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.0)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))